In [ ]:
# Training v1 SPY Prediction Model on Google Colab (Free)

This notebook trains your v1 SPY prediction model by securely exposing your local PostgreSQL (TimescaleDB) database to Colab using ngrok (free tier). Follow these steps carefully.

**Key Details from Your Setup:**
- **Database Port**: 5433 (confirmed by Docker container `tsdb` running `timescale/timescaledb-ha`).
- **Database Name**: `trading_data` (as shown in pgAdmin).
- **Tables**: Located in the `backtest` schema (e.g., `backtest.spy_1m`, `backtest.spy_5m`, etc.).
- **TimescaleDB**: Your database uses the `timescaledb` extension, optimized for time-series data.
- **Connection Method**: ngrok tunnel (free tier - no payment required).


In [ ]:
## Step 1: Prerequisites (Local Machine)
- **Install ngrok**: 
  - Go to [ngrok.com](https://ngrok.com) and create a free account
  - Download ngrok for your operating system
  - Extract and place in your PATH, or note the location
- **Get ngrok authtoken**: 
  - Login to your ngrok dashboard
  - Copy your authtoken from the "Your Authtoken" section
- **Configure ngrok**: Run `ngrok config add-authtoken <your_token>` on your local machine
- **Docker Confirmation**: Your TimescaleDB container (`tsdb`) is running on port 5433, as seen in Docker Desktop.
- **No router configuration needed**: ngrok handles all networking automatically!


In [ ]:
## Step 2: Start ngrok tunnel (Local Machine)
Start the ngrok tunnel on your local machine to expose your database:


In [ ]:
On your LOCAL MACHINE, open a terminal/command prompt and run:

ngrok tcp 5433

This will create a tunnel to your local database on port 5433.
Copy the forwarding address (something like tcp://0.tcp.ngrok.io:12345)
You'll need this for the database connection in the next steps.

Example output:
Session Status    online
Forwarding        tcp://0.tcp.ngrok.io:12345 -> localhost:5433

Keep this terminal open during the entire training process!


In [ ]:
## Step 3: Get ngrok connection details
After starting ngrok on your local machine, you should see output like:
```
Forwarding        tcp://0.tcp.ngrok.io:12345 -> localhost:5433
```

- Copy the hostname: `0.tcp.ngrok.io`
- Copy the port: `12345`
- You'll use these in the database connection below


In [ ]:
# No code needed here - the ngrok tunnel runs on your local machine
# Make sure you have started ngrok with: ngrok tcp 5433
# and copied the forwarding URL before proceeding to the next step
print("Ensure ngrok is running on your local machine: ngrok tcp 5433")


In [ ]:
## Step 4: Prepare ngrok connection details
From your ngrok output, extract the connection details:

**Example ngrok output:**
```
Forwarding        tcp://0.tcp.ngrok.io:12345 -> localhost:5433
```

**Extract these values:**
- **Host**: `0.tcp.ngrok.io` (everything before the colon)
- **Port**: `12345` (everything after the colon)

You'll use these values in the database connection string in the next step.


In [ ]:
## Step 5: Install Database Libraries


In [ ]:
!pip install sqlalchemy psycopg2-binary pandas --quiet


In [ ]:
## Step 6: Connect to Database via ngrok
- Update the credentials (`user`, `password`) to match your local PostgreSQL setup.
- Replace `NGROK_HOST` and `NGROK_PORT` with the values from your ngrok output.
- The connection uses the ngrok tunnel to reach your local database.


In [ ]:
from sqlalchemy import create_engine

# Replace with your actual database credentials and ngrok details
# Get NGROK_HOST and NGROK_PORT from your ngrok output (e.g., 0.tcp.ngrok.io and 12345)
NGROK_HOST = "0.tcp.ngrok.io"  # Replace with your ngrok host
NGROK_PORT = "12345"           # Replace with your ngrok port
DB_USER = "user"               # Replace with your database username
DB_PASSWORD = "password"       # Replace with your database password

# Create connection string using ngrok tunnel
connection_string = f'postgresql://{DB_USER}:{DB_PASSWORD}@{NGROK_HOST}:{NGROK_PORT}/trading_data'
engine = create_engine(connection_string)

print(f"Connecting to: {NGROK_HOST}:{NGROK_PORT}")

# Test connection
try:
    with engine.connect() as connection:
        result = connection.execute("SELECT 1")
        print("Connection successful:", result.fetchone())
        print("✅ Database connection established via ngrok!")
except Exception as e:
    print(f"❌ Connection failed: {e}")
    print("Make sure ngrok is running and the credentials are correct")


In [ ]:
## Step 7: Load SPY Data from `backtest` Schema
- Loads data for all timeframes into a dictionary of DataFrames.
- Queries are updated to use the `backtest` schema (e.g., `SELECT * FROM backtest.spy_1m`).


In [ ]:
import pandas as pd

timeframes = ['1m', '5m', '15m', '30m', '1h', '4h', '1d']
data = {}

for tf in timeframes:
    query = f"SELECT * FROM backtest.spy_{tf}"
    data[tf] = pd.read_sql_query(query, engine)
    print(f"Loaded {tf} data: {data[tf].shape}")


In [ ]:
## Step 8: Upload and Train Your Model
- **Option 1**: Upload `spy_prediction_model_v1.py` via the Colab file upload UI, then import it.
- **Option 2**: Copy the contents of `spy_prediction_model_v1.py` into the cell below.
- Assumes your pipeline is in a function called `run_full_pipeline`.


In [ ]:
# Option 1: If uploaded
from spy_prediction_model_v1 import run_full_pipeline

# Run the pipeline
run_full_pipeline(data)


In [ ]:
## Step 9: Monitor and Close the Tunnel
- Keep this notebook running during training.
- After training, stop ngrok by pressing Ctrl+C in the terminal running `ngrok tcp 5433`.
- The ngrok tunnel will automatically close when you stop the process.


In [ ]:
## Notes
- **Security**: Ensure your database credentials and ngrok authtoken are secure.
- **Performance**: Ensure a stable internet connection for data transfer.
- **Free Usage**: ngrok free tier provides secure tunneling without payment requirements.
- **Troubleshooting**: If the connection fails, verify:
  - ngrok tunnel is active (`ngrok tcp 5433` running).
  - The Docker container (`tsdb`) is running.
  - Your database credentials are correct.
  - The ngrok host and port are correctly updated in the connection string.
- **Schema**: Tables are explicitly queried from the `backtest` schema.
- **TimescaleDB**: Your database is optimized for time-series data, ensuring efficient queries for large datasets.
- **ngrok Benefits**: No router configuration, automatic HTTPS, and works from anywhere.
